In [1]:
import time
import threading
from threading import Thread
import random
import sys

## Shared Data and Synchronization

As we saw in our previous notebook, threads can access to share data. This is useful to communicate things and access common variables. But it'll also introduce problems, mainly Race Conditions. Let's see an example of a Race Condition.

In [2]:
COUNTER = 0

In [3]:

def increment(n):
    global COUNTER
    for _ in range(n):
        COUNTER += 1
        time.sleep(0.001)

In [4]:

ITERATIONS = 1000

In [5]:
threads = [Thread(target=increment, args=(ITERATIONS,)) for _ in range(10)]

In [6]:
[t.start() for t in threads];

In [7]:
assert COUNTER == (len(threads) * ITERATIONS), f"Invalid value for counter: {COUNTER}"

AssertionError: Invalid value for counter: 9060

Both threads are executing concurrently (potentially at the same time) and they're reading outdated values of COUNTER, which results in a race condition.

## Thread Synchronization

How can we fix then this race condition? Basically, we need a way to keep the threads from stepping onto each other's data, some signal that the given resource "is busy":

![title](thread.png)

(Example of INEs studios, a recording light is on, the studio is busy, nobody will enter the room)

The easiest synchronization mechanism is a Lock), or a Mutex (mutual exclusion lock). Python includes the very intuitive threading.Lock class. Let's see how a Lock works.

A Lock works in the same way as the Studio Light from the picture. The first one that "arrives" to that given resource "turns on the light", or, formally, "acquires the lock". Any other threads reaching that point, if they want to acquire the lock, they have to wait for the first thread to "release it". Let's see an example:

## Locking

In [28]:
lock = threading.Lock()

In [29]:
def lock_hogger(lock, sleep=10):
    print("\t\tThread: Acquiring lock.")
    lock.acquire()
    print("\t\tThread: Lock acquired, sleeping")
    if sleep:
        time.sleep(sleep)
    print("\t\tThread: Woke up, releasing lock")
    lock.release()

In [30]:
t = Thread(target=lock_hogger, args=(lock, ))

In [31]:
t.start()

		Thread: Acquiring lock.
		Thread: Lock acquired, sleeping


In [32]:
lock.locked()

True

In [33]:
lock.acquire()
print("Lock acquired!")

		Thread: Woke up, releasing lock
Lock acquired!


In [34]:
lock.release()

## Time to fix our counter!

Now that we know about locks, we can use them to fix our counter example:

In [59]:
COUNTER=0

In [60]:

def increment(n,lock):
    global COUNTER
    for _ in range(n):
        lock.acquire()
        COUNTER += 1
        lock.release()
        time.sleep(0.001)

In [61]:
lock = threading.Lock()

In [62]:
ITERATIONS = 1000

In [63]:
threads = [Thread(target=increment, args=(ITERATIONS, lock)) for _ in range(10)]

In [64]:
[t.start() for t in threads];

In [65]:
[t.join() for t in threads];

In [66]:

COUNTER

10000

In [67]:
assert COUNTER == (len(threads) * ITERATIONS), f"Invalid value for counter: {COUNTER}"

## It doesn't matter how many times we run the example, our code will always be synchronized!

## Problems with synchronization

Locks are acquired before accessing what we call "Critical Sections"; important sections in our code that can potentially introduce race conditions. The usual process is:

In [69]:
lock = threading.Lock() # Problem [1]

# before entering critical section
lock.acquire()          # Problem [2]

# critical section
# do_your_thing()         # Problem [3]

# after we're done with it
lock.release()          # Problem [4]

The problem is that locks (and many other synchronization mechanisms) are "cooperative". You're cooperating by using locks, but you're not obliged to use them. In a team of n developers, just one of them screws up with their lock management, everybody loses.

These are the things that can potentially go wrong with cooperative, manual synchronization mechanisms:

You might forget to use locks at all! You might have failed to recognize the situation as having a "critical section".
You might forget to acquire the lock, getting directly into the critical section.
Your critical section might be using resources NOT protected by the lock you're using, so other threads can be stepping onto that.
You might forget to release the lock, or your code could break before you're able to release the lock (next example)
Deadlocks! (more on later).
Problem No.4 is very common, let's see an example of it: